In [1]:
# Dependencies
import tweepy
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta
from datetime import timezone 
from pprint import pprint
import time
import re

In [2]:
# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

# Twitter API Keys
from config import (consumer_key, 
                    consumer_secret, 
                    access_token, 
                    access_token_secret)

# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser(),wait_on_rate_limit=True)

In [ ]:
target_terms = ("@CNN", "@ABC", "@CBSNews", "@FoxNews", "@NBCNews", "@nytimes")
# Variables for holding sentiments
sentiments = []
for target in target_terms:
    # Counter
    counter = 1
    # Variable for max_id
    oldest_tweet = None
    # Loop through 5 pages of tweets (total 100 tweets)
    for x in range(6):
        public_tweets = api.user_timeline(target, count=200, tweet_mode='extended',max_id = oldest_tweet)
        for tweet in public_tweets:
            results = analyzer.polarity_scores(tweet["full_text"])
            compound = results["compound"]
            pos = results["pos"]
            neu = results["neu"]
            neg = results["neg"]
            tweets_ago = counter
            # Get Tweet ID, subtract 1, and assign to oldest_tweet
            oldest_tweet = tweet['id'] - 1
            sentiments.append({"News Account": target,
                               "News Text": re.sub(r'https*?:\/\/.*[\r]*','',tweet["full_text"],flags=re.IGNORECASE),
                               "News Created Date": tweet["created_at"], 
                               "News id": tweet['id'],
                               "News Account id": tweet['user']['id'],
                               "Retweet Count": tweet['retweet_count'],
                               "Like Count": tweet['favorite_count'],
                               "Followers Count": tweet['user']['followers_count'],
                               "News Compound": compound,
                               "News Positive": pos,
                               "News Neutral": neu,
                               "News Negative": neg,
                               "News Tweets Ago": counter})
            counter += 1

In [ ]:
# Convert sentiments to DataFrame
news_sentiments_pd = pd.DataFrame.from_dict(sentiments)
news_sentiments_pd = news_sentiments_pd[["News Account",
                                    "News Text",
                                    "News Created Date",
                                    "News id",
                                    "News Account id",
                                    "Retweet Count",
                                    "Like Count",
                                    "Followers Count",
                                    "News Compound",
                                    "News Positive",
                                    "News Neutral",
                                    "News Negative",
                                    "News Tweets Ago"]]

# export sentiments dataframe to csv
news_sentiments_pd.to_csv("News_Twitter.csv")

In [12]:
def tweet_reply(media,tweet_id):
    search_term = media
    oldest_tweet = tweet_id + 1000000000000
    sentiments = []
    counter = 1
    for x in range(2):
        try:
            public_tweets = api.search(search_term, count=100,tweet_mode='extended', 
                                   since_id = tweet_id, max_id = oldest_tweet)
        except Exception:
            time.sleep(60)
            continue
        for tweet in public_tweets["statuses"]:
            if tweet["in_reply_to_status_id"] == tweet_id:
                results = analyzer.polarity_scores(tweet["full_text"])
                compound = results["compound"]
                pos = results["pos"]
                neu = results["neu"]
                neg = results["neg"]
                tweets_ago = counter
                # Get Tweet ID, subtract 1, and assign to oldest_tweet
                oldest_tweet = tweet['id'] - 1
                sentiments.append({"News Account": media,
                                   "News id": tweet_id,
                                   "Reply Text": re.sub(r'https*?:\/\/.*[\r]*','',tweet["full_text"],flags=re.IGNORECASE),
                                   "Reply Created Date": tweet["created_at"], 
                                   "Reply Tweet id": tweet['id'],
                                   "Reply User id": tweet['user']['id'],
                                   "Reply Compound": compound,
                                   "Reply Positive": pos,
                                   "Reply Neutral": neu,
                                   "Reply Negative": neg,
                                   "Reply Tweets Ago": counter})
                print(f"{counter} reply tweets found, {tweet['full_text']}, {tweet['created_at']}")
                counter += 1               
    return sentiments   

In [10]:
csv_file = "News_Twitter CNN.csv"
news_twitter = pd.read_csv(csv_file)
news_twitter = news_twitter.drop(["Unnamed: 0"],axis=1)
reply_sentiment = []
for index, row in news_twitter.iterrows():
    news_account = row["News Account"]
    news_id = row["News id"]
    reply_sentiment = reply_sentiment + tweet_reply(news_account,news_id)

1 reply tweets found, @CNN ❤️❤️❤️, Mon Apr 16 00:21:42 +0000 2018
2 reply tweets found, @CNN I don't see how Russia and a bombing committed by Kyrgyzstani terrorists in the name of Islam are in any way related, but maybe the nuance and genius of this particular act of performativity are lost on me., Mon Apr 16 00:21:31 +0000 2018
3 reply tweets found, @CNN What happened to space between b and o, Mon Apr 16 00:20:23 +0000 2018
4 reply tweets found, @CNN That is so awesome!, Mon Apr 16 00:19:41 +0000 2018
5 reply tweets found, @CNN Let's create a federal holiday for every terror event that the FBI screwed up.  We'll never have to work again., Mon Apr 16 00:19:20 +0000 2018
6 reply tweets found, @CNN Cool, Mon Apr 16 00:19:15 +0000 2018
1 reply tweets found, @CNN @CNNTravel 1 match and poof 🔥🔥🔥, Mon Apr 16 00:10:38 +0000 2018
2 reply tweets found, @CNN @CNNTravel Step 1) gather bamboo, Mon Apr 16 00:08:58 +0000 2018
3 reply tweets found, @CNN @CNNTravel That's some incredible architecture

11 reply tweets found, @CNN Quick! Somebody get him a Rasmussen!!, Sun Apr 15 22:17:44 +0000 2018
12 reply tweets found, @CNN FAKE NEWS, Sun Apr 15 22:17:38 +0000 2018
13 reply tweets found, @CNN Democrats=useless, Sun Apr 15 22:17:27 +0000 2018
14 reply tweets found, @CNN If it weren’t for ignorance it would be lower than it is. Trump has no business trying to act like a president. He doesn’t know what he is doing., Sun Apr 15 22:17:19 +0000 2018
15 reply tweets found, @CNN Dang i didnt think it was that high!, Sun Apr 15 22:17:15 +0000 2018
16 reply tweets found, @CNN Jim @Comey 📚 : @BarackObama and Loretta #Lynch 'jeopardized' the @HillaryClinton email investigation. Comey I understand that. It was obvious! Having said that @FBI is an Independent agency. You had the power to indict Clinton! You had evidence! #Tatiana
https://t.co/Nkq5q2JzvM, Sun Apr 15 22:17:11 +0000 2018
17 reply tweets found, @CNN Now do Rasmussen and quinnipiac. Ya kno, the polls that people actually give a shit 

8 reply tweets found, @CNN Those 3 countries are the Air Force and Marine of the terrorist groups. They supported them from the start and now that Syria, Russia and Iran are winning against their allies, the felt they had to come to the rescue! So pathetic!, Sun Apr 15 20:47:03 +0000 2018
9 reply tweets found, @CNN https://t.co/GuPNYZEJdm, Sun Apr 15 20:46:17 +0000 2018
10 reply tweets found, @CNN https://t.co/4pZAJH0hKV, Sun Apr 15 20:46:09 +0000 2018
11 reply tweets found, @CNN It simply means risk on us being terrorised has increased, Sun Apr 15 20:46:09 +0000 2018
12 reply tweets found, @CNN Puuf, Sun Apr 15 20:46:08 +0000 2018
13 reply tweets found, @CNN Unfortunately nothing.  It needs to be much more aggressive for there to be any long term effect., Sun Apr 15 20:45:55 +0000 2018
14 reply tweets found, @CNN Easy, it means https://t.co/HAJENijU9e, Sun Apr 15 20:45:49 +0000 2018
1 reply tweets found, @CNN For what purpose? I suspect there's more to this than protesting for gay rig

9 reply tweets found, @CNN Now l know Apofure Onojovwo is a coward if he cannot give me justice against my own father., Sun Apr 15 19:17:26 +0000 2018
10 reply tweets found, @CNN An Australian Muslim activist was deported from the US.
#FakeFreedom #fakedemocracy #stopwars #ThisIsAmerica #AmericanRacism #BoycottAmerica #BoycottIsrael #BDS https://t.co/ddGtdlJBbv, Sun Apr 15 19:17:12 +0000 2018
11 reply tweets found, @CNN Yeh but will Trump in force them?????, Sun Apr 15 19:17:03 +0000 2018
12 reply tweets found, @CNN  https://t.co/2C7GKwh47L, Sun Apr 15 19:16:47 +0000 2018
1 reply tweets found, @CNN .

Total partisan hack.....

Thank god Trump kicked his big butt out of the SDNY office with no hesitation !!!

. https://t.co/RAZgzgzoYP, Sun Apr 15 19:04:47 +0000 2018
2 reply tweets found, @CNN Ya think there is some bias here.☹️, Sun Apr 15 19:03:49 +0000 2018
3 reply tweets found, @CNN https://t.co/fSezv2pMEi, Sun Apr 15 19:03:35 +0000 2018
4 reply tweets found, @CNN Only if Trump doesn

8 reply tweets found, @CNN Pual Ryan dipped out after shitting in the pool and robbing the cash register. Good luck suckers lolol aynrand4lyf, Sun Apr 15 16:46:54 +0000 2018
9 reply tweets found, @CNN No follow up on this? 

Fire at Trump Tower leaves man dead and 6 firefighters injured

https://t.co/Fz9CjFTWiq, Sun Apr 15 16:46:51 +0000 2018
10 reply tweets found, @CNN Fake news, Sun Apr 15 16:46:33 +0000 2018
1 reply tweets found, @CNN Sad. All this needless war., Sun Apr 15 16:32:52 +0000 2018
2 reply tweets found, @CNN my my the timing... gosh.. what a surprise, Sun Apr 15 16:32:42 +0000 2018
3 reply tweets found, @CNN How many tunnels can be funded by Mexican drugsmoney?, Sun Apr 15 16:32:22 +0000 2018
4 reply tweets found, @CNN https://t.co/fSezv2pMEi, Sun Apr 15 16:32:18 +0000 2018
5 reply tweets found, @CNN Another one will be open soon., Sun Apr 15 16:31:38 +0000 2018
1 reply tweets found, @CNN @jaketapper  https://t.co/PAjrjyIQ1v, Sun Apr 15 16:18:40 +0000 2018
2 reply tweets

...

15 reply tweets found, @CNN @npwcnn I’ll never be able to trust that CNN isn’t using a green screen when a reporter is on assignment #fakenews, Sat Apr 14 11:39:03 +0000 2018
1 reply tweets found, @CNN Think I lost 5 years last night. Those damn Coronas., Sat Apr 14 11:34:54 +0000 2018
2 reply tweets found, @CNN I don't drink, but I do believe you should live for the moment. You could be hit by a bus tomorrow so by not drinking doesn't mean a thing except you will see it coming more clearly!, Sat Apr 14 11:34:48 +0000 2018
3 reply tweets found, @CNN Perhaps. But I’ll take the chance as I choose quality of life. And your statement counters most research, especially on red wine., Sat Apr 14 11:34:41 +0000 2018
4 reply tweets found, @CNN See?, Sat Apr 14 11:34:39 +0000 2018
5 reply tweets found, @CNN With trump in office i could give a shit less, Sat Apr 14 11:34:32 +0000 2018
6 reply tweets found, @CNN Yeah, but it may also make it more livable  so it's one of those situations, Sat Apr 1

4 reply tweets found, @CNN  https://t.co/eY9vcoNiPH, Sat Apr 14 10:22:06 +0000 2018
5 reply tweets found, @CNN Americans like war so much I hope that the United States will have 9/11 events every day, Sat Apr 14 10:21:36 +0000 2018
6 reply tweets found, @CNN The view from Damascus. 

https://t.co/DTDzGJyVmr, Sat Apr 14 10:21:17 +0000 2018
1 reply tweets found, @CNN inspectors were on the ground, was the site of the chemical attack bombed? CNN should find out, Sat Apr 14 10:16:04 +0000 2018
2 reply tweets found, @CNN fake news + fake wars = cnn, Sat Apr 14 10:15:38 +0000 2018
3 reply tweets found, @CNN I'm old enough to remember when Trump was supposed to be some sort of lackey for Putin. So much for that grand theory., Sat Apr 14 10:15:06 +0000 2018
4 reply tweets found, @CNN So we have 3 countries dropping bombs, and yet...we see no destruction? Syria must have shot down some like they said. This was wagging the dog. Wait a week, tell them ur plans, then limit ur airstrikes? Trump is 

11 reply tweets found, @CNN @CNNOpinion You should pay us for wasting our time reading this shit., Sat Apr 14 09:02:20 +0000 2018
1 reply tweets found, @CNN A truly pathetic show from a terrified prime minister who can’t stray from rehearsed answers., Sat Apr 14 09:01:03 +0000 2018
2 reply tweets found, @CNN Then why the fuck didn't you take part in the tomahawk launching that trump did on the Syrian airbase before., Sat Apr 14 09:00:52 +0000 2018
3 reply tweets found, @CNN WRONG ON BOTH ILEGAL ACTS AGAIN FROM ROGO STATS 
AND YOU THE MEDIA LET THEM DO THIS 
WAKE UP SORT YOUR CRAP OUT STOP THESE STUPID PEOPLE  KILL OTHERS, Sat Apr 14 09:00:34 +0000 2018
4 reply tweets found, @CNN How about you attack the country that strolled in and poisoned people in your own Homeland. Not that brave, eh? Just expell some more diplomats, I'm sure that will solve it., Sat Apr 14 09:00:04 +0000 2018
5 reply tweets found, @CNN @SharonP47713509 the only person that even comes close to being as inept and us

1 reply tweets found, @CNN @StCollinson @WhalePack, Sat Apr 14 07:27:08 +0000 2018
2 reply tweets found, @CNN @StCollinson Every body know he is a poor sick man, Sat Apr 14 07:26:33 +0000 2018
3 reply tweets found, @CNN @StCollinson Yes.  @realDonaldTrump has a heart.  Do some real reporting.  There's nothing unconventional about saving babies from the chemical baths their country's leader wants to inflict.  I'd say that was damn heroic., Sat Apr 14 07:25:42 +0000 2018
4 reply tweets found, @CNN @StCollinson Do not justify his actions. They were clearly done as a diversion. We are living Wag the Dog., Sat Apr 14 07:25:20 +0000 2018
5 reply tweets found, @CNN @StCollinson I want to fuck @realDonaldTrump, Sat Apr 14 07:24:59 +0000 2018
6 reply tweets found, @CNN @StCollinson Any incident could instantly blow up. Both sides #USA and #Russia are modernizing and increasing their military systems, including nuclear systems., Sat Apr 14 07:24:05 +0000 2018
1 reply tweets found, @CNN #Syria Th

TweepError: Failed to send request: ('Connection aborted.', OSError("(60, 'ETIMEDOUT')",))

In [11]:
reply_sentiments_pd = pd.DataFrame.from_dict(reply_sentiment)
reply_sentiments_pd = reply_sentiments_pd[["News Account",
                                   "News id",
                                   "Reply Text",
                                   "Reply Created Date", 
                                   "Reply Tweet id",
                                   "Reply User id",
                                   "Reply Compound",
                                   "Reply Positive",
                                   "Reply Neutral",
                                   "Reply Negative",
                                   "Reply Tweets Ago"]]

# export sentiments dataframe to csv
reply_sentiments_pd.to_csv("Reply_Twitter_CNN.csv")

In [13]:
csv_file = "News_Twitter CNN copy.csv"
news_twitter = pd.read_csv(csv_file)
news_twitter = news_twitter.drop(["Unnamed: 0"],axis=1)
reply_sentiment = []
for index, row in news_twitter.iterrows():
    news_account = row["News Account"]
    news_id = row["News id"]
    reply_sentiment = reply_sentiment + tweet_reply(news_account,news_id)

1 reply tweets found, @CNN @cueckert Here we go🤯, Sat Apr 14 06:35:42 +0000 2018
2 reply tweets found, @CNN  https://t.co/vonDCSOKv3, Sat Apr 14 06:34:40 +0000 2018
3 reply tweets found, @CNN Yup America we are gonna lose troops for no reason... Again, Sat Apr 14 06:34:25 +0000 2018
4 reply tweets found, @CNN US warships,  B-1 bombers strike against Assad after suspected chemical attack https://t.co/jzA9pAqZTa https://t.co/sVWo7mNy0i, Sat Apr 14 06:33:57 +0000 2018
5 reply tweets found, @CNN Anything to distract from Stormy, Comey and PeePee. #syria, Sat Apr 14 06:33:46 +0000 2018
6 reply tweets found, @CNN . haiiyya pake bahasa indonesia mayyy, Sat Apr 14 06:33:37 +0000 2018
7 reply tweets found, @CNN Is it also to make America great again?! 😒😡, Sat Apr 14 06:33:35 +0000 2018
8 reply tweets found, @CNN The United States is the Godfather when it establishes edicts. Others had better live up to them, or else. We have to demonstrate that. So that's what the bombing of #Syria was to have 

1 reply tweets found, @CNN Turns out they just want bomb the chemical attack scene of the crime before investigators get there tomorrow.

#SaveSyria
https://t.co/KPuOYkWk6D, Sat Apr 14 05:44:59 +0000 2018
2 reply tweets found, @CNN welcome to the swamp mr. trump, Sat Apr 14 05:44:56 +0000 2018
3 reply tweets found, @CNN According to current international law, any military action launched against a sovereign state without the authorization of the UN is illegal and can be defined as aggression., Sat Apr 14 05:44:41 +0000 2018
4 reply tweets found, @CNN well I'm about to go to bed but know this I am praying to God that Donald Trump is no longer with us when I get up in the morning Justice must be served hell is waiting for that scumbag, Sat Apr 14 05:44:34 +0000 2018
5 reply tweets found, @CNN #WagTheDog He was tired of people talking about him &amp; @MichaelCohen212, Sat Apr 14 05:44:25 +0000 2018
6 reply tweets found, @CNN If I print this, it won't be to read it. It will be because I ra

6 reply tweets found, @CNN No wait, Trump doing this for bad news cover-up.  Wait, what!!! (as Russian-bot head explodes), Sat Apr 14 04:20:30 +0000 2018
7 reply tweets found, @CNN WWIII has began again led failed politicians, Sat Apr 14 04:20:29 +0000 2018
8 reply tweets found, @CNN THERE WAS NO FUCKING PROOF. You fucking idiots are going to get us all killed, they could have waited for the investigation results instead of bombing first facts later. Iraq, Afghanistan all these other stupid fucking wars, you guys are playing with Fire. Putin is not a pussy., Sat Apr 14 04:20:27 +0000 2018
9 reply tweets found, @CNN just now reporting this? I read this hours ago., Sat Apr 14 04:20:26 +0000 2018
10 reply tweets found, @CNN Hmm, Sat Apr 14 04:20:24 +0000 2018
1 reply tweets found, @CNN @realDonaldTrump Unless Trump takes out Assad, this is just another deflect and distract event to divert attention from Russiagate, Stormygate and Trump's crooked regime ..., Sat Apr 14 04:05:05 +0000 2018


1 reply tweets found, @CNN Message to Russia.....Take Trump back with you...if you like him so much, Sat Apr 14 02:52:09 +0000 2018
2 reply tweets found, @CNN The United States has no Secretary of State and we just attacked another country.  Trump is a nightmare, Sat Apr 14 02:51:59 +0000 2018
3 reply tweets found, @CNN  https://t.co/E5TvTI7VEp, Sat Apr 14 02:51:49 +0000 2018
4 reply tweets found, @CNN Lol what’s Russia going to do about it?, Sat Apr 14 02:51:47 +0000 2018
5 reply tweets found, @CNN I am taking off my hat and bow to President Trump!, Sat Apr 14 02:51:44 +0000 2018
6 reply tweets found, @CNN Thought Putin was our buddy, Sat Apr 14 02:51:38 +0000 2018
7 reply tweets found, @CNN https://t.co/1QR9CohuIs, Sat Apr 14 02:51:35 +0000 2018
8 reply tweets found, @CNN Fk the Russian ambassador. He needs to become fertilizer along with Assad, Putin and every other EVIL tyrant., Sat Apr 14 02:51:28 +0000 2018
9 reply tweets found, @CNN Suck it, Sat Apr 14 02:51:22 +0000 2018
10 rep

1 reply tweets found, @CNN There goes that Russia collusion theory, Sat Apr 14 01:31:53 +0000 2018
2 reply tweets found, @CNN Good job Trump, get clean the chemical attack that's kill childrens, Sat Apr 14 01:31:48 +0000 2018
3 reply tweets found, @CNN Yeah. After trump signaled what we were going to do so that the Syrians and the Russians could move equipment. Are we bombing empty airfields again?? @CNN @MSNBC @FoxNews wagthedog, Sat Apr 14 01:31:31 +0000 2018
4 reply tweets found, @CNN Same retards loving this, got a hard on bombing Iraq. Cue idiotic redneck song., Sat Apr 14 01:31:24 +0000 2018
1 reply tweets found, @CNN Yes - we are going to die. GoIng to kiss that beautiful woman I've been admiring., Sat Apr 14 01:27:35 +0000 2018
2 reply tweets found, @CNN They let 45 launch a air strike? OMG! I gotta watch CNN!, Sat Apr 14 01:27:12 +0000 2018
3 reply tweets found, @CNN Good that Trump built a coalition. Kudos to him. Pray that the troops come home safe., Sat Apr 14 01:27:05 +000

...

10 reply tweets found, @CNN 😕, Sun Apr 08 04:03:53 +0000 2018
11 reply tweets found, @CNN Well...cant say I blame them, Sun Apr 08 04:03:52 +0000 2018
12 reply tweets found, @CNN Play stupid games Win stupid prizes, Sun Apr 08 04:03:37 +0000 2018
13 reply tweets found, @CNN Looks like suicide by cop to me., Sun Apr 08 04:03:24 +0000 2018
14 reply tweets found, @CNN Liberals will still defend this man, Sun Apr 08 04:03:09 +0000 2018
15 reply tweets found, @CNN  https://t.co/rFwuNTSNeq, Sun Apr 08 04:03:05 +0000 2018
16 reply tweets found, @CNN Interesting situation, Sun Apr 08 04:03:03 +0000 2018
17 reply tweets found, @CNN Lesson: Don't point objects at police that look like guns. https://t.co/EMvGjQ9mF3, Sun Apr 08 04:02:55 +0000 2018
18 reply tweets found, @CNN Good. You don't play like that., Sun Apr 08 04:02:53 +0000 2018
19 reply tweets found, @CNN Coo coo. Coo coo, Sun Apr 08 04:02:42 +0000 2018
1 reply tweets found, @CNN It's Divisionist Nancy who should have resigned long ago.,

9 reply tweets found, @CNN Bernies bots, aka Russia is back. Do not lose focus, Trump won from Bernie last time, Sun Apr 08 02:01:54 +0000 2018
10 reply tweets found, @CNN The left is eating itself. Chow down, commies. You won't get much to eat when the top commie puts you in the gulag., Sun Apr 08 02:01:39 +0000 2018
11 reply tweets found, @CNN JUST IN: Liberal blue checks celebrate as Trump Tower fire leaves 1 dead, 4 injured, Sun Apr 08 02:01:33 +0000 2018
12 reply tweets found, @CNN Doesn’t matter. NY will never be a red state, Sun Apr 08 02:01:22 +0000 2018
13 reply tweets found, @CNN  https://t.co/rZpRH0Oh9M, Sun Apr 08 02:00:55 +0000 2018
1 reply tweets found, @CNN Ban assault lightning, Sun Apr 08 01:38:39 +0000 2018
2 reply tweets found, @CNN Wow, that is crazy! So sad, Sun Apr 08 01:38:25 +0000 2018
3 reply tweets found, @CNN #FakeNews   Verify with Fox before you believe anything you see here., Sun Apr 08 01:38:21 +0000 2018
4 reply tweets found, @CNN 😞🙏🏻, Sun Apr 08 01:37:5

11 reply tweets found, @CNN your forgetting that there are kids that spilled their blood and don't agree with the kids that did not get blood on their cloths and are reading your narrative., Sun Apr 08 00:22:49 +0000 2018
12 reply tweets found, @CNN yeah its weird how we never hear from the victims or their families but just that same group of students #FaknewsCNN https://t.co/DVuihLBJxU, Sun Apr 08 00:22:03 +0000 2018
1 reply tweets found, @CNN This is sad., Sun Apr 08 00:21:51 +0000 2018
2 reply tweets found, @CNN Sad pray for em., Sun Apr 08 00:21:18 +0000 2018
3 reply tweets found, @CNN Liberalism is a mental disorder., Sun Apr 08 00:21:09 +0000 2018
4 reply tweets found, @CNN This story is so just so unbelievably sad.  Those poor children.  😢  May they find eternal peace and comfort in the arms of Jesus., Sun Apr 08 00:21:06 +0000 2018
5 reply tweets found, @CNN God Bless this Family, Sun Apr 08 00:21:03 +0000 2018
1 reply tweets found, @CNN Whiskey, Sun Apr 08 00:14:06 +0000 2018

8 reply tweets found, @CNN I love how #fakenewsCNN does all this investigating about who knew of wikileaks instead of actually investigating the content of the leaks https://t.co/Ha7TIXiHqb, Sat Apr 07 23:16:53 +0000 2018
9 reply tweets found, @CNN Roger Stone in Trump Tower smoking meth about to burn down the whole building., Sat Apr 07 23:16:51 +0000 2018
10 reply tweets found, @CNN Shut up and pray that you survive the great awakening https://t.co/Ckq7MEDXDD, Sat Apr 07 23:16:48 +0000 2018
11 reply tweets found, @CNN ... watch one interview with Stone and you can size him up pretty quick..., Sat Apr 07 23:16:37 +0000 2018
12 reply tweets found, @CNN Very good android app to save any VIDEOS / GIF from twitter 💪
check my tweets to get it, Sat Apr 07 23:15:52 +0000 2018
13 reply tweets found, @CNN https://t.co/CbprS9qSMX, Sat Apr 07 23:15:48 +0000 2018
1 reply tweets found, @CNN @VanJones68 America is a great Country and the biggest democracy of the world....Yes, America will survive d

12 reply tweets found, @CNN The Germans dindu nuffins wrong, Sat Apr 07 22:32:05 +0000 2018
13 reply tweets found, @CNN What a wonderful public speaker., Sat Apr 07 22:32:01 +0000 2018
14 reply tweets found, @CNN  https://t.co/tfsI4YKCYi, Sat Apr 07 22:31:49 +0000 2018
15 reply tweets found, @CNN No., Sat Apr 07 22:31:42 +0000 2018
16 reply tweets found, @CNN https://t.co/JBNftednyH, Sat Apr 07 22:31:33 +0000 2018
17 reply tweets found, @CNN Showing images of Hitler is extremely insensitive!, Sat Apr 07 22:31:22 +0000 2018
18 reply tweets found, @CNN Sellout, Sat Apr 07 22:31:04 +0000 2018


In [14]:
reply_sentiments_pd = pd.DataFrame.from_dict(reply_sentiment)
reply_sentiments_pd = reply_sentiments_pd[["News Account",
                                   "News id",
                                   "Reply Text",
                                   "Reply Created Date", 
                                   "Reply Tweet id",
                                   "Reply User id",
                                   "Reply Compound",
                                   "Reply Positive",
                                   "Reply Neutral",
                                   "Reply Negative",
                                   "Reply Tweets Ago"]]

# export sentiments dataframe to csv
reply_sentiments_pd.to_csv("Reply_Twitter_CNN_copy.csv")

In [15]:
csv_file = "News_Twitter CBS.csv"
news_twitter = pd.read_csv(csv_file)
news_twitter = news_twitter.drop(["Unnamed: 0"],axis=1)
reply_sentiment = []
for index, row in news_twitter.iterrows():
    news_account = row["News Account"]
    news_id = row["News id"]
    reply_sentiment = reply_sentiment + tweet_reply(news_account,news_id)

1 reply tweets found, @CBSNews Allegiant pilot? https://t.co/bzUXc833M1, Mon Apr 16 00:24:31 +0000 2018
2 reply tweets found, @CBSNews This is the worst airline in the country., Mon Apr 16 00:23:00 +0000 2018
3 reply tweets found, @CBSNews Fake news cbs 
Fake news cbs

Nobody died!, Mon Apr 16 00:22:39 +0000 2018
4 reply tweets found, @CBSNews Typical 60 seconds hit piece ... one sided, Mon Apr 16 00:22:07 +0000 2018
5 reply tweets found, @CBSNews Good., Mon Apr 16 00:22:04 +0000 2018
1 reply tweets found, @CBSNews Unbelievable that we still know very little about this shooting and why this tragedy happened., Mon Apr 16 00:12:35 +0000 2018
2 reply tweets found, @CBSNews They're so brave, Mon Apr 16 00:12:07 +0000 2018
3 reply tweets found, @CBSNews My mom always watched her show. I thought I was pretty good., Mon Apr 16 00:11:52 +0000 2018
4 reply tweets found, @CBSNews Watching #ACMawards Great opening of the program. 💕, Mon Apr 16 00:11:09 +0000 2018
1 reply tweets found, @CBSNews Gr

1 reply tweets found, @CBSNews 😂😂😂, Sun Apr 15 18:03:04 +0000 2018
2 reply tweets found, @CBSNews Now you're reporting on the feelings of pandas?, Sun Apr 15 18:02:19 +0000 2018
3 reply tweets found, @CBSNews  https://t.co/WkhQPmVtOL, Sun Apr 15 18:02:08 +0000 2018
4 reply tweets found, @CBSNews ...sooooo cute!, Sun Apr 15 18:01:56 +0000 2018
1 reply tweets found, @CBSNews #narcos, Sun Apr 15 17:43:54 +0000 2018
2 reply tweets found, @CBSNews Too bad for Pablo Escobar, this would of been very beneficial for him!, Sun Apr 15 17:42:01 +0000 2018
1 reply tweets found, @CBSNews I’ll cross that anxiety off my list then, Sun Apr 15 17:25:24 +0000 2018
2 reply tweets found, @CBSNews HE MUST HAVE SOMETHING REALLY IMPORTANT TO DO IN HIS LIFE., Sun Apr 15 17:25:21 +0000 2018
3 reply tweets found, @CBSNews "I can fly, I can fly, I can fly"...Fine, but tell him not to make a career out of it, things may not go so well next time., Sun Apr 15 17:24:12 +0000 2018
4 reply tweets found, @CBSNews What k

1 reply tweets found, @CBSNews Or we could just do this civil war thing. These fucks with the guns think they’d win... ok. Let’s go., Sun Apr 15 14:24:22 +0000 2018
2 reply tweets found, @CBSNews Here come all the crying white supremacists. Nope. You're not oppressed. Go form your own social platform., Sun Apr 15 14:23:53 +0000 2018
3 reply tweets found, @CBSNews Planned parenthood, Sun Apr 15 14:23:37 +0000 2018
4 reply tweets found, @CBSNews NRA next please, Sun Apr 15 14:22:57 +0000 2018
1 reply tweets found, @CBSNews  https://t.co/8N6pbnoY48, Sun Apr 15 14:04:23 +0000 2018
2 reply tweets found, @CBSNews https://t.co/mJnPF57tTN, Sun Apr 15 14:04:16 +0000 2018
3 reply tweets found, @CBSNews Remind anyone of a reaction from - let's say - a high school student?  What a pathetic POTUS we have., Sun Apr 15 14:03:35 +0000 2018
4 reply tweets found, @CBSNews IS THIS REAL LIFE, Sun Apr 15 14:03:30 +0000 2018
5 reply tweets found, @CBSNews #WeBelieveComey, Sun Apr 15 14:03:16 +0000 2018
6 re

1 reply tweets found, @CBSNews The generation with access to the most information knows the least., Sun Apr 15 10:21:58 +0000 2018
2 reply tweets found, @CBSNews 4 in 10 millennials believe Trump is literally Hitler, social media shows., Sun Apr 15 10:21:36 +0000 2018
3 reply tweets found, @CBSNews And how many know that 25/26 million Russians died defeating Hitler? That would be interesting to know., Sun Apr 15 10:20:42 +0000 2018
4 reply tweets found, @CBSNews https://t.co/lGPEP9Ao1q, Sun Apr 15 10:20:12 +0000 2018
5 reply tweets found, @CBSNews 4 in 10 ... 

Sounds like the conservative side..., Sun Apr 15 10:20:00 +0000 2018
6 reply tweets found, @CBSNews Or care., Sun Apr 15 10:19:57 +0000 2018
7 reply tweets found, @CBSNews 6 of 10 millennials do know about the Holocaust - that is the better statistic., Sun Apr 15 10:19:53 +0000 2018
1 reply tweets found, @CBSNews It was about 5 years ago this week somebody was found dismembered in a trash can....at our softball field, right befo

1 reply tweets found, @CBSNews Nice more censorship., Sun Apr 15 04:14:00 +0000 2018
2 reply tweets found, @CBSNews There is no “free speech” for these kinds of people., Sun Apr 15 04:12:41 +0000 2018
1 reply tweets found, @CBSNews More reason for the elderly to tip like shit., Sun Apr 15 04:06:49 +0000 2018
2 reply tweets found, @CBSNews While we are at it. Maybe Congress before passing new laws(etc) they should take a #drugs #test, Sun Apr 15 04:06:33 +0000 2018
3 reply tweets found, @CBSNews I’m sure coke use is even higher in the hospitality industry 

Give us a story we care about. Waiters smoking pot? 🤷🏾‍♀️ Just bring me my snacks., Sun Apr 15 04:06:20 +0000 2018
4 reply tweets found, @CBSNews 🤔 https://t.co/XGI441jbzt, Sun Apr 15 04:05:54 +0000 2018
5 reply tweets found, @CBSNews  https://t.co/DIspCFduH2, Sun Apr 15 04:05:33 +0000 2018
6 reply tweets found, @CBSNews Could have told ya that for free, Sun Apr 15 04:05:12 +0000 2018
7 reply tweets found, @CBSNews They needed to do 

...

1 reply tweets found, @CBSNews He’s suing., Sat Apr 07 21:18:35 +0000 2018
2 reply tweets found, @CBSNews Still lying about this exposed staged event CBS? You lying traitors. https://t.co/clxK1Ij3bh, Sat Apr 07 21:17:17 +0000 2018
3 reply tweets found, @CBSNews So did the @FBI so far nobody had been fired for this grave failure. Remember this..., Sat Apr 07 21:17:06 +0000 2018
4 reply tweets found, @CBSNews Don't say that!  It doesn't align with their agendas!, Sat Apr 07 21:16:39 +0000 2018
1 reply tweets found, @CBSNews I'm sure almost all of the employees of the @EPA would be more than happy to turn over absolutely anything to get rid of @EPAScottPruitt., Sat Apr 07 20:58:42 +0000 2018
2 reply tweets found, @CBSNews Why not report on the pedophile site being shut down?, Sat Apr 07 20:57:49 +0000 2018
3 reply tweets found, @CBSNews Mr. President, this man must be stopped from squandering the taxpayer's dollars. It's outrageous. Are you listening?, Sat Apr 07 20:57:28 +0000 2018
4 rep

1 reply tweets found, @CBSNews Tell Trump..., Sat Apr 07 17:17:34 +0000 2018
2 reply tweets found, @CBSNews All safe all that matters, Sat Apr 07 17:17:09 +0000 2018
3 reply tweets found, @CBSNews TRUMP #infrastructure fail, Sat Apr 07 17:17:08 +0000 2018
4 reply tweets found, @CBSNews https://t.co/nHZOfCR3RK, Sat Apr 07 17:16:24 +0000 2018
1 reply tweets found, @CBSNews The Amish?, Sat Apr 07 16:59:14 +0000 2018
2 reply tweets found, @CBSNews https://t.co/OCB1wA0svl, Sat Apr 07 16:58:27 +0000 2018
3 reply tweets found, @CBSNews Islamo-nazi terrorists will eventually take over Europe and leftist media is helping them, Sat Apr 07 16:58:13 +0000 2018
4 reply tweets found, @CBSNews https://t.co/nHZOfCR3RK, Sat Apr 07 16:55:44 +0000 2018
1 reply tweets found, @CBSNews that's three training crashes in less than a week... two choppers and an #F16, Sat Apr 07 16:43:43 +0000 2018
2 reply tweets found, @CBSNews Sad, Sat Apr 07 16:42:38 +0000 2018
1 reply tweets found, @CBSNews Islamic terrorist

1 reply tweets found, @CBSNews Well we are screwed because of Trump, Sat Apr 07 14:21:32 +0000 2018
2 reply tweets found, @CBSNews https://t.co/nHZOfCR3RK, Sat Apr 07 14:20:35 +0000 2018
1 reply tweets found, @CBSNews What a loser coach !!!, Sat Apr 07 14:01:25 +0000 2018
1 reply tweets found, @CBSNews Callie should have full reservoirs for the summer so they can go back to watering their yards and washing cars., Sat Apr 07 13:42:13 +0000 2018
1 reply tweets found, @CBSNews Heartbreaking., Sat Apr 07 13:21:53 +0000 2018
2 reply tweets found, @CBSNews So very sad.prayers for all those involved, Sat Apr 07 13:20:40 +0000 2018
1 reply tweets found, @CBSNews Really sad story, hope they find his killer, Sat Apr 07 13:05:43 +0000 2018
2 reply tweets found, @CBSNews heinous criminal, Sat Apr 07 13:04:53 +0000 2018
3 reply tweets found, @CBSNews The killing needs to stop, Sat Apr 07 13:04:50 +0000 2018
4 reply tweets found, @CBSNews By who? Who shot him?!😑😐, Sat Apr 07 13:04:22 +0000 2018
1 re

1 reply tweets found, @CBSNews @feralislandBud So Sad https://t.co/TtPX1WrlsE, Sat Apr 07 06:05:43 +0000 2018
2 reply tweets found, @CBSNews Hmm. Trump will be on the case., Sat Apr 07 06:05:02 +0000 2018
1 reply tweets found, @CBSNews Gilligan!, Sat Apr 07 05:48:50 +0000 2018
1 reply tweets found, @CBSNews I’d do her, Sat Apr 07 05:37:01 +0000 2018
1 reply tweets found, @CBSNews 😪😫😢, Sat Apr 07 05:19:32 +0000 2018
1 reply tweets found, @CBSNews yawn, Sat Apr 07 05:06:46 +0000 2018
2 reply tweets found, @CBSNews fake news., Sat Apr 07 05:06:46 +0000 2018
3 reply tweets found, @CBSNews  https://t.co/ifaknnviAH, Sat Apr 07 05:06:30 +0000 2018
4 reply tweets found, @CBSNews No more adult supervision in the @WhiteHouse . It's #LordoftheFlies time., Sat Apr 07 05:06:10 +0000 2018
5 reply tweets found, @CBSNews Baby https://t.co/ied4sHE5qC, Sat Apr 07 05:06:05 +0000 2018
6 reply tweets found, @CBSNews No. That’s not bratty at all., Sat Apr 07 05:06:00 +0000 2018
7 reply tweets found, @CBSNew

1 reply tweets found, @CBSNews FB is breaching user data while hiding it's own., Sat Apr 07 01:23:10 +0000 2018
2 reply tweets found, @CBSNews They've been selling your private information to the highest bidder who use this information to create a dossier to attack their enemies. The board of directors need to be arrested. Now I understand how #Facebook make their money., Sat Apr 07 01:21:21 +0000 2018
1 reply tweets found, @CBSNews It’s California they will be back on street by midnight, Sat Apr 07 00:58:17 +0000 2018
2 reply tweets found, @CBSNews Hope it was jp1.  Its just kerosene. And not nearly as flammable or expensive as avgas, Sat Apr 07 00:58:09 +0000 2018
3 reply tweets found, @CBSNews Some more of @JerryBrownGov kids doing what they do.., Sat Apr 07 00:57:26 +0000 2018
4 reply tweets found, @CBSNews SAN FRAN  WILL WANT TO PROTECT THEM., Sat Apr 07 00:56:59 +0000 2018
1 reply tweets found, @CBSNews  https://t.co/mVKUJYwRGY, Sat Apr 07 00:38:32 +0000 2018
2 reply tweets found

In [16]:
reply_sentiments_pd = pd.DataFrame.from_dict(reply_sentiment)
reply_sentiments_pd = reply_sentiments_pd[["News Account",
                                   "News id",
                                   "Reply Text",
                                   "Reply Created Date", 
                                   "Reply Tweet id",
                                   "Reply User id",
                                   "Reply Compound",
                                   "Reply Positive",
                                   "Reply Neutral",
                                   "Reply Negative",
                                   "Reply Tweets Ago"]]

# export sentiments dataframe to csv
reply_sentiments_pd.to_csv("Reply_Twitter_CBS.csv")

In [18]:
csv_file = "News_Twitter NBC.csv"
news_twitter = pd.read_csv(csv_file)
news_twitter = news_twitter.drop(["Unnamed: 0"],axis=1)
reply_sentiment = []
for index, row in news_twitter.iterrows():
    news_account = row["News Account"]
    news_id = row["News id"]
    reply_sentiment = reply_sentiment + tweet_reply(news_account,news_id)

1 reply tweets found, @NBCNews Comey interview tonight @  10pm on ABC. The Truth Will Set You Free!, Mon Apr 16 00:18:08 +0000 2018
2 reply tweets found, @NBCNews How about a trial against the US government and @realDonaldTrump for war crimes and terrorism in Syria?, Mon Apr 16 00:16:36 +0000 2018
3 reply tweets found, @NBCNews A life sentence for being a Christian in a so called allied country. Absurd., Mon Apr 16 00:16:19 +0000 2018
1 reply tweets found, @NBCNews Anti birth control advocates weighing in on 3...2...1..., Mon Apr 16 00:01:30 +0000 2018
2 reply tweets found, @NBCNews Why would you put a spring in your vagina?, Mon Apr 16 00:01:09 +0000 2018
1 reply tweets found, @NBCNews @MSNBC Or tune into "Trump's Top Ten Slimeballs!" on the Fox News Channel at 9 pm. If the President hates them so do we!, Sun Apr 15 23:46:12 +0000 2018
2 reply tweets found, @NBCNews @MSNBC Comy is a disgrace to himself, his family and his country.  He should be hanged., Sun Apr 15 23:45:40 +0000 2018


1 reply tweets found, @NBCNews Alot of our problems should be uniting No one should want a gubment that spies and lies doesnt respect our laws., Sun Apr 15 21:33:04 +0000 2018
2 reply tweets found, @NBCNews NEW NEW WORLD ORDER

"Silicon Valley isn’t just disrupting democracy—it’s replacing it " @qz

#ZuckerbergHearings #Dystopia https://t.co/chnX9AMsce, Sun Apr 15 21:32:23 +0000 2018
3 reply tweets found, @NBCNews Because no one wants their dirty laundry out for everyone to see.....cause everyone’s got something they want to keep quiet., Sun Apr 15 21:32:14 +0000 2018
4 reply tweets found, @NBCNews L. O. L, Sun Apr 15 21:31:44 +0000 2018
5 reply tweets found, @NBCNews https://t.co/NnXFLxTkvW, Sun Apr 15 21:31:41 +0000 2018
6 reply tweets found, @NBCNews What unites us is that we all hate Zuckerberg., Sun Apr 15 21:31:16 +0000 2018
7 reply tweets found, @NBCNews Whatever it takes...this is ridiculous!, Sun Apr 15 21:30:59 +0000 2018
1 reply tweets found, @NBCNews Off course it is black 

12 reply tweets found, @NBCNews it's good to be @POTUS ... hope the weather is decent..., Sun Apr 15 17:00:05 +0000 2018
13 reply tweets found, @NBCNews https://t.co/fSezv2pMEi, Sun Apr 15 16:59:18 +0000 2018
1 reply tweets found, @NBCNews @TODAYshow Cool them and you’ll be fine...., Sun Apr 15 17:00:27 +0000 2018
2 reply tweets found, @NBCNews @TODAYshow https://t.co/fSezv2pMEi, Sun Apr 15 16:58:19 +0000 2018
1 reply tweets found, @NBCNews Lol why don’t you ask De Niro what he has done after hours behind closed doors that have pinholes in masking tape in SOHO ijs, Sun Apr 15 16:31:00 +0000 2018
2 reply tweets found, @NBCNews Prime example of why I don’t watch the TV show., Sun Apr 15 16:30:54 +0000 2018
3 reply tweets found, @NBCNews That was funny the first 500 times eighteen years ago. https://t.co/Zu3AiR7I1s, Sun Apr 15 16:30:52 +0000 2018
4 reply tweets found, @NBCNews One can argue that Ben Stiller ruined De Niro’s career (terrible awful comedies one after another)., Sun Apr 15 1

1 reply tweets found, @NBCNews @MeetThePress Ryan is a POS.  Bye Felicia, Sun Apr 15 13:43:56 +0000 2018
2 reply tweets found, @NBCNews @MeetThePress Goodbye sycophant Ryan, don't let the doorknob hit you in the azz on your way out. Mueller needs to be protected and if the Republicans don't do it then they will be removed from power at the mid-terms., Sun Apr 15 13:43:33 +0000 2018
3 reply tweets found, @NBCNews @MeetThePress Says the spineless weasel., Sun Apr 15 13:43:03 +0000 2018
4 reply tweets found, @NBCNews @MeetThePress The only reason that I can think of for why Ryan, McConnell and the GOP refuse to #ProtectMueller is that they are afraid their crimes have been uncovered, right along with Trump's.  #TrumpIsNotAboveTheLaw   #SundayMorning, Sun Apr 15 13:42:56 +0000 2018
5 reply tweets found, @NBCNews @MeetThePress Complicit a-hole!, Sun Apr 15 13:42:48 +0000 2018
6 reply tweets found, @NBCNews @MeetThePress Shut up POS Russian butt lover. You’re all will be gone in a few months

1 reply tweets found, @NBCNews Hotel check in men are suppose to get you "anything" you want that isn't crime against humanity.  It's their job., Sun Apr 15 05:02:28 +0000 2018
2 reply tweets found, @NBCNews That’s because they’re tired of your crap, Barbara., Sun Apr 15 05:02:22 +0000 2018
3 reply tweets found, @NBCNews Shocking, Sun Apr 15 05:02:17 +0000 2018
4 reply tweets found, @NBCNews No shit!, Sun Apr 15 05:02:13 +0000 2018
5 reply tweets found, @NBCNews Potheads, Sun Apr 15 05:01:20 +0000 2018
1 reply tweets found, @NBCNews @NBCNewsTHINK Well you don't have to be a cheerleader., Sun Apr 15 03:02:50 +0000 2018
2 reply tweets found, @NBCNews @NBCNewsTHINK Look no further than Buffalo. They haven't had the Jills for a few years now. No pay, sexual harassment, etc..., Sun Apr 15 03:02:19 +0000 2018
3 reply tweets found, @NBCNews @NBCNewsTHINK Haha,cheer on, Sun Apr 15 03:01:57 +0000 2018
4 reply tweets found, @NBCNews @NBCNewsTHINK Do YOU mean bitchs?, Sun Apr 15 03:01:48 +0000 20

...

1 reply tweets found, @NBCNews Wad it a high capacity van?, Sat Apr 07 16:31:21 +0000 2018
2 reply tweets found, @NBCNews More 'Religion Of Peace', Sat Apr 07 16:30:33 +0000 2018
3 reply tweets found, @NBCNews We need to ban assault vans!, Sat Apr 07 16:30:20 +0000 2018
4 reply tweets found, @NBCNews When will Merkel step up and remove the occupying jihadists? @AfD_Tweets needs to win in Germany to restore its greatness!, Sat Apr 07 16:29:59 +0000 2018
5 reply tweets found, @NBCNews Ban the Van, Sat Apr 07 16:29:53 +0000 2018
1 reply tweets found, @NBCNews The system is driving everyone crazy it is part of the NWO agenda..., Sat Apr 07 16:22:13 +0000 2018
2 reply tweets found, @NBCNews Time for Germany  to put it's guard on their borders, Sat Apr 07 16:21:55 +0000 2018
3 reply tweets found, @NBCNews another Mohammed, Sat Apr 07 16:20:28 +0000 2018
4 reply tweets found, @NBCNews Hmm, I wonder which group did this? Any guesses?, Sat Apr 07 16:20:12 +0000 2018
5 reply tweets found, @NBCNe

1 reply tweets found, @NBCNews So what? Let’s keep all out whiskey! I’ll volunteer to drink all the @JackDaniels_US, Sat Apr 07 12:32:20 +0000 2018
2 reply tweets found, @NBCNews #FakeNews Not one #Tariff has been implemented yet!, Sat Apr 07 12:32:11 +0000 2018
1 reply tweets found, @NBCNews @NBCNewsBETTER https://t.co/TSDTKB7Waq, Sat Apr 07 12:00:58 +0000 2018
1 reply tweets found, @NBCNews @MSNBC Of course he is. They buy his shit!, Sat Apr 07 11:33:24 +0000 2018
2 reply tweets found, @NBCNews @MSNBC I support DACA. My daughter has a friend who is a DACA recipient... She graduated college with a tech degree and is currently employed paying taxes and contributing so much. She's scared every day. She did what the country asked and now she's being betrayed. It's so sad., Sat Apr 07 11:32:17 +0000 2018
3 reply tweets found, @NBCNews @MSNBC Reason #498 to not buy Apple products.  The first 497 are because I can handle a device that isn't still designed for people with no technical skills

1 reply tweets found, @NBCNews Thanks Donny!  We can always rely on you to create new problems where none previously existed., Sat Apr 07 05:16:31 +0000 2018
1 reply tweets found, @NBCNews On a positive note, this means people are still reading the classics and interested enough to visit., Sat Apr 07 05:03:02 +0000 2018
2 reply tweets found, @NBCNews Good.  Now they can just call it Piss Lake, Sat Apr 07 05:02:34 +0000 2018
3 reply tweets found, @NBCNews I hate people., Sat Apr 07 05:02:18 +0000 2018
4 reply tweets found, @NBCNews I guess aquatic animals don't pee or poo. Who knew?, Sat Apr 07 05:02:10 +0000 2018
1 reply tweets found, @NBCNews https://t.co/CCZqveQJVh, Sat Apr 07 04:45:08 +0000 2018
1 reply tweets found, @NBCNews @NBCLatino The trump administration refuses to help Puerto Rico yet Pruitt can spend millions on 20 security guards, have a 25k sound proof room in his office and fly first class on our dime, Sat Apr 07 04:33:55 +0000 2018
2 reply tweets found, @NBCNews @NBCLat

8 reply tweets found, @NBCNews True ....or we could just stop giving tax money to corporations that pay 0 taxes and post world record profits each year.  But, I guess paying more for all of our favorite stuff is just as good. #eyeroll, Sat Apr 07 01:47:14 +0000 2018
9 reply tweets found, @NBCNews Just don’t bring that ish to Chicago, Sat Apr 07 01:46:52 +0000 2018
10 reply tweets found, @NBCNews It’s the poor who mostly purchase the crap, Sat Apr 07 01:46:41 +0000 2018
1 reply tweets found, @NBCNews @ScottPruittOK is a paranoid freak because he’s destroying the environment and selling out to corporations, Sat Apr 07 01:43:44 +0000 2018
2 reply tweets found, @NBCNews Even with death threats he still has the balls to rip apart the Environmental protection agency. Whelp when your old and don’t have to deal with the consequences.. so what. This guy is horrible, Sat Apr 07 01:43:29 +0000 2018
3 reply tweets found, @NBCNews The fiscal irresponsibility within this administration is astounding

1 reply tweets found, @NBCNews @NBCNightlyNews Awesome, Sat Apr 07 00:03:53 +0000 2018
2 reply tweets found, @NBCNews @NBCNightlyNews Dam it I use to get my escorts on that site!, Sat Apr 07 00:03:15 +0000 2018
1 reply tweets found, @NBCNews For those who are activated thank you very much for your service🇺🇸Hopefully he’ll get bored with playing with all of you soon 🙄 https://t.co/wikxFYwM3w, Fri Apr 06 23:52:56 +0000 2018
2 reply tweets found, @NBCNews To do . . . Nothing.  Except let @POTUS dementia Donnie play army., Fri Apr 06 23:52:45 +0000 2018
3 reply tweets found, @NBCNews  https://t.co/4xEPB3ntVk, Fri Apr 06 23:52:34 +0000 2018
4 reply tweets found, @NBCNews What a waste of time and resources while Puerto Rico still suffers and Flint doesnt have clean water and and and. Americans can't even take care of the basics, yet they do this., Fri Apr 06 23:52:24 +0000 2018
5 reply tweets found, @NBCNews Cool now what's going on on the border of Gaza and Israel can happen on the border o

In [19]:
reply_sentiments_pd = pd.DataFrame.from_dict(reply_sentiment)
reply_sentiments_pd = reply_sentiments_pd[["News Account",
                                   "News id",
                                   "Reply Text",
                                   "Reply Created Date", 
                                   "Reply Tweet id",
                                   "Reply User id",
                                   "Reply Compound",
                                   "Reply Positive",
                                   "Reply Neutral",
                                   "Reply Negative",
                                   "Reply Tweets Ago"]]

# export sentiments dataframe to csv
reply_sentiments_pd.to_csv("Reply_Twitter_NBC.csv")